In [15]:
%run ../Data-Preprocessing.ipynb
%run ../U-Net.ipynb

import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [5]:
convert_ct_dataset_to_slices('Task06_Lung', 'Lung_Train', 'Lung_Val', 'Lung_Test', val_split=0.1, test_split=0.1, negative_downsampling_rate=5)

In [16]:
transform = transforms.Compose([
    transforms.Resize((128, 128), antialias=False),
    transforms.ConvertImageDtype(torch.float)
])

train_images = os.path.join('Lung_Train', 'images')
train_labels = os.path.join('Lung_Train', 'labels')

train_dataset = CTDataset(train_images, train_labels, image_transform=transform, label_transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

val_images = os.path.join('Lung_Val', 'images')
val_labels = os.path.join('Lung_Val', 'labels')

val_dataset = CTDataset(val_images, val_labels, image_transform=transform, label_transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

test_images = os.path.join('Lung_Test', 'images')
test_labels = os.path.join('Lung_Test', 'labels')

test_dataset = CTDataset(test_images, test_labels, image_transform=transform, label_transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=os.cpu_count())

In [17]:
model = UNet()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
num_epochs = 50

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs} started...")
    
    train_running_loss = 0.
    valid_running_loss = 0.

    model.train()
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad() 
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for images, masks in validation_loader:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            valid_running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{num_epochs} end...')
    print(f'Training loss: {train_running_loss / len(train_loader)}')
    print(f'Validation loss: {valid_running_loss / len(validation_loader)}')
    print('...........................................')

Epoch 1/50 started...


KeyboardInterrupt: 